In [4]:
using ForwardDiff

In [5]:
f(x) = 3*x^2
ForwardDiff.derivative(f,3) # berechnet die Ableitung von f(x) an der Stelle 3

18

In [7]:
function f!(y,x)
    y[1] = 3*x^2
    y[2] = x^4
    nothing
end

y = [0,0]
f!(y,3)
@show y

y = [0,0]
ForwardDiff.derivative(f!, y, 3) #Syntax .derivative(funktion, Variable, Stelle der Ableitung)

y = [27, 81]


2-element Vector{Int64}:
  18
 108

In [10]:
g(x) = 3*x[1]^2+x[2]^4
g([1,2])
ForwardDiff.gradient(g,  [1,2])

h(x,p) = p[1] *x[1]^2 +p[2]*x[2]^4
h([1,2], [3,1])
ForwardDiff.gradient(x -> h(x, [3,1]),[1,2])

2-element Vector{Int64}:
  6
 32

Reverse Differentiation

In [12]:
using ReverseDiff

In [16]:
g(x) = 3*x[1]^2+x[2]^4
g([1,2])
ReverseDiff.gradient(g,  [1,2])

h(x,p) = p[1] *x[1]^2 +p[2]*x[2]^4
h([1,2], [3,1])
ReverseDiff.gradient(x -> h(x, [3,1]),[1,2])

2-element Vector{Int64}:
 0
 0

In [19]:
using Zygote
f(x) = 3*x+5
f(10), f'(10)

(35, 3.0)

In [20]:
using Flux

In [1]:
using Flux
using Flux.Data.MNIST
using Plots

# Load MNIST data
_, test_data = MNIST.traindata()

# Load the trained model (assuming you've already trained a model)
model = load("mnist_model.bson")  # Replace with your model file name

# Define a function to get misclassified examples
function misclassified_examples(model, data)
    misclassified = []
    for (x, y) in data
        pred = Flux.onecold(model(x))
        true_label = Flux.onecold(y)
        if pred != true_label
            push!(misclassified, (x, true_label, pred))
        end
    end
    return misclassified
end

misclassified_data = misclassified_examples(model, test_data)

# Visualize some of the misclassified examples
num_examples_to_visualize = 10
plot_array = []
for (x, true_label, pred) in misclassified_data[1:num_examples_to_visualize]
    push!(plot_array, heatmap(reshape(x, 28, 28), color=:grays, title="True: $true_label, Predicted: $pred"))
end

plot(plot_array..., layout=(2, 5))

UndefVarError: UndefVarError: `MNIST` not defined

In [9]:
using ReverseDiff

# Define a simple function
f(x) = x^2 + sin(x)

# Define the input value
x = 2.0

# Compute the gradient
gradient_result = gradient(f, x)

println("Gradient at x = $x: $gradient_result")

Gradient at x = 2.0: (3.5838531634528574,)


In [10]:
using ForwardDiff

# Define a simple function
f(x) = x^2 + sin(x)

# Define the input value
x = 2.0

# Compute the gradient
gradient_result = ForwardDiff.derivative(f, x)

println("Gradient at x = $x: $gradient_result")

Gradient at x = 2.0: 3.5838531634528574


REVERSEDIFF FASTER THAN FORWARDDIFF

In [27]:
using ReverseDiff
using BenchmarkTools

# Define a simple function
f(x) = sum(x .^ 2)

# Input values
x = rand(1000)

# Benchmark ReverseDiff (backward differentiation)
@btime gradient(f, x)

# Benchmark ForwardDiff (forward differentiation)
using ForwardDiff
@btime ForwardDiff.gradient(f, x)

  7.175 μs (18 allocations: 16.28 KiB)


  2.076 ms (173 allocations: 8.44 MiB)


1000-element Vector{Float64}:
 1.4404907492776222
 0.3641136609090274
 0.6578963057858984
 0.860395528110609
 1.2044421778501537
 1.2142689380070693
 0.3290150644767269
 1.278188466856067
 1.7407254585747634
 1.9545773140781482
 ⋮
 0.5680727783008848
 1.706572468315645
 0.66622831666345
 1.19005750345353
 1.0382907953611165
 1.0958346723238006
 0.16621766339843091
 0.5207973292796837
 0.5442212367900865

µs: Microsekunden 1µs = 1*10^-6 s
ms: Millisekunden 1ms = 1*10^-3 s
ReverseDiff: 7*10^-6s = 0,000007s
ForwardDiff: 2,049*10^-3s = 0,002049s

In [39]:
using ReverseDiff
using BenchmarkTools

# Define a simple function
f(x) = sum(x .^ 2)

# Input values
x = [1,2,3,4,5]

# Benchmark ReverseDiff (backward differentiation)
ableitungsrev = gradient(f,x)
@btime ableitungsrev


# Benchmark ForwardDiff (forward differentiation)
using ForwardDiff
ableitungsforw = ForwardDiff.gradient(f, x)
@btime ableitungsforw
 
ableitungsrev == ableitungforw

  5.300 ns (0 allocations: 0 bytes)


  5.300 ns (0 allocations: 0 bytes)


false

In [24]:
f(x::Vector) = sin(x[1]/x[2])
x = vcat(1.5,0.5)
f(x)

0.1411200080598672

In [25]:
using ForwardDiff
forwardmode = ForwardDiff.gradient(f(x),x)

MethodError: MethodError: objects of type Float64 are not callable
Maybe you forgot to use an operator such as *, ^, %, / etc. ?

REVERSE MODE VS. FORWARD MODE

In [4]:
using BenchmarkTools
f(x::Vector) = sin(x[1]) + prod(x[2:end]);  # returns a scalar
x = vcat(pi/4, 2:4)
f(x)

using Zygote
@btime reversemode = gradient(f,x)

using ForwardDiff
@btime forwardmode = ForwardDiff.gradient(f, x)

  2.244 μs (25 allocations: 1.94 KiB)


  863.158 ns (5 allocations: 784 bytes)


4-element Vector{Float64}:
  0.7071067811865476
 12.0
  8.0
  6.0

In diesem Fall ist reverse mode marginal langsamer als forward mode, denn 
Input-Dimension = 2
Output-Dimension = 1
Es lässt sich erst ein deutlich größerer Unterschied feststellen, wenn man eine komplexere Funktion nimmt.

In [33]:
reversemode == forwardmode
# ergibt false, da forwardmode ist ein Vektor und reversemode ist ein Tupel

false

In [3]:
using BenchmarkTools
f(x) = sum(x .^ 2)
x = rand(200)

using Zygote
@btime reversemode = gradient(f,x)

using ForwardDiff
@btime forwardmode = ForwardDiff.gradient(f, x)

  3.938 μs (25 allocations: 8.66 KiB)


  128.800 μs (39 allocations: 369.45 KiB)


200-element Vector{Float64}:
 0.6897428282430618
 2.4547265011722097e-89
 2.9624250017012377e-90
 2.0517947668614588e-89
 1.1207899158386772e-88
 4.186187577940991e-90
 2.903385333185479e-90
 1.2516275403209047e-89
 4.353442887045468e-90
 2.6330113309392543e-90
 ⋮
 3.4492258051058264e-90
 9.491555107190879e-90
 1.0606667675090935e-89
 5.317354348370391e-90
 2.972852054540731e-90
 3.071795504853622e-90
 2.5381557352798786e-89
 6.292523738436738e-90
 4.864819594381441e-90

Hier sieht man schon, dass bei deutlich mehr Daten der reverse mode schneller ist, als der forward mode.
Und hier ist auch Input-Dimension=200 >> Ouput-Dimension=1

In [48]:
using BenchmarkTools
using Zygote
@time hessian(f,x)

using ForwardDiff
@time ForwardDiff.hessian(f,x)

  0.001475 seconds (496 allocations: 2.431 MiB)


 28.747069 seconds (10.40 M allocations: 696.440 MiB, 1.46% gc time, 99.60% compilation time)


200×200 Matrix{Float64}:
 -0.784689  0.0          0.0          …  0.0          0.0
  0.0       0.0          2.57242e-87     6.12671e-87  2.32851e-87
  0.0       2.57242e-87  0.0             7.18441e-87  2.7305e-87
  0.0       1.23169e-87  1.44433e-87     3.43994e-87  1.30738e-87
  0.0       4.89912e-87  5.7449e-87      1.36826e-86  5.20018e-87
  0.0       1.18848e-87  1.39365e-87  …  3.31925e-87  1.26151e-87
  0.0       5.71095e-85  6.69688e-85     1.59499e-84  6.0619e-85
  0.0       1.80537e-86  2.11704e-86     5.04214e-86  1.91631e-86
  0.0       1.20455e-87  1.4125e-87      3.36414e-87  1.27857e-87
  0.0       3.25753e-87  3.81991e-87     9.09784e-87  3.45772e-87
  ⋮                                   ⋱               
  0.0       1.22461e-87  1.43602e-87     3.42016e-87  1.29986e-87
  0.0       2.01864e-87  2.36714e-87     5.63778e-87  2.14269e-87
  0.0       9.66942e-87  1.13387e-86     2.70053e-86  1.02636e-86
  0.0       4.87885e-87  5.72113e-87     1.36259e-86  5.17866e-87
  0.0 

AUTODIFF: BACKPROPAGATION

AUTODIFF IN LAYERS
Trainieren des neuronalen Netzes, indem die Gewichte verbessert werden. Das geschieht durch Anwendung des Gradientenverfahrens, denn es muss das Minimum der Funktion E gefunden werden. 

In [ ]:
input = [0,1,2,3,4,5,6,7,8,9]
ouput = [0,1,2,3,4,5,6,7,8,9]
gewichtsmatrix = randn()

In [52]:
using MaNN
# Struktur des neuronalen Netzes
# implementiere jeweils 4 Gewichts- und 4 Biasmatrizen
mutable struct NeuronalNetwork
    W1::Matrix{Float64}
    b1::Vector{Float64}
    W2::Matrix{Float64}
    b2::Vector{Float64}
    W3::Matrix{Float64}
    b3::Vector{Float64}
    W4::Matrix{Float64}
    b4::Vector{Float64}
end

In [53]:
# Funktion, die die Gewichts- und Biasmatrizen in ihrer jeweiligen Größe mit zufälligen Werten initialisiert
# W hat so viele Zeilen wie Neuronen im nächsten hidden layer sind und so viele Spalten wie die Anzahl der Inputs ist
# b ist ein Vektor mit so vielen Zeilen wie Neuronen in dem nächsten hidden layer sind
function initialize_network(input_size, hidden_size1, hidden_size2, output_size)
    W1 = randn(hidden_size1, input_size)
    b1 = zeros(hidden_size1)
    W2 = randn(hidden_size2, hidden_size1)
    b2 = zeros(hidden_size2)
    W3 = randn(output_size, hidden_size2)
    b3 = zeros(output_size)
    W4 = randn(output_size, hidden_size2)
    b4 = zeros(output_size)
    return NeuralNetwork(W1, b1, W2, b2, W3, b3)
end

initialize_network (generic function with 1 method)

In [55]:
#Daten müssen irgendwie noch umgeändert werden, denn das funktioniert so nicht, da trainset nicht einfach nur 
# ein array oder int ist
data = 1

using Zygote

function backpropagate!(model, data, y_true, learning_rate)
    # Forward pass
    # zi ist die Summe der gewichteten Eingangsdaten
    # yi ist der hidden state, also die Aktivierungsfunktion auf die zi angewandt
    z1 = NeuronalNetwork.W1 * data .+ NeuronalNetwork.b1
    y1 = MaNN.leakyrelu(z1)
    z2 = NeuronalNetwork.W2 * y1 .+ NeuronalNetwork.b2
    y2 = MaNN.leakyrelu(z2)
    z3 = NeuronalNetwork.W3 * y2 .+ NeuronalNetwork.b3
    y3 = MaNN.leakyrelu(z3)
    y4 = MaNN.softmax(y3)

    # Compute loss
    # a4 ist der Output des neuronalen Netzes
    loss = MaNN.cross_entropy(y_true, y4)

    # Backpropagation
    ## unbedingt mit zygote(reversediff) schreiben
    ableitungW4 = gradient(loss, W4)
    W4 = NeuronalNetwork.W4 - learning_rate*ableitungW4
    ableitungW3 = gradient(loss, W3)
    W3 = NeuronalNetwork.W3 - learning_rate*ableitungW3
    ableitungW2 = gradient(loss, W2)
    W2 = NeuronalNetwork.W2 - learning_rate*ableitungW2
    ableitungW1 = gradient(loss, W1)
    W1 = NeuronalNetwork.W1 - learning_rate*ableitungW1
    return loss
end

backpropagate! (generic function with 1 method)

In [ ]:
function train_hardcoded(model, data, labels, learning_rate, num_epochs)
    for epoch in 1:num_epochs
        total_loss = 0
        for i in 1:size(trainset, 2)
            x = trainset[:,i]
            y_true = labels[:,i]
            loss = backporpagate!(model, x, y_true, learning_rate)
            total_loss += loss
        end
        println("Epoch:" +epoch+ "Loss:" +total_loss)
    end
end

In [43]:
input_size = 784
hidden_size1 = 256
hidden_size2 = 128
output_size = 10
labels = [i == j ? 1.0:0.0 for i in 1:output_size, j in 1:100]
learning_rate = 0.1
num_epochs = 10
train_hardcoded(model, data, labels, learning_rate, num_epochs)

ErrorException: syntax: space required before colon in "?" expression